# Install jupyter

```
pip install jupyter
```



# Jupyter spark enviroment.
```
export PYSPARK_DRIVER_PYTHON=jupyter
export PYSPARK_DRIVER_PYTHON_OPTS="notebook"
pyspark --master local[2] --packages graphframes:graphframes:0.3.0-spark1.6-s_2.11 
```

# Get file list

In [1]:
import requests
import lxml.html as lh

gdelt_base_url = 'http://data.gdeltproject.org/events/'

page = requests.get(gdelt_base_url+'index.html')
doc = lh.fromstring(page.content)
link_list = doc.xpath("//*/ul/li/a/@href")

file_list = [x for x in link_list if str.isdigit(x[0:4])]

In [2]:
import datetime
datetime.datetime.now().strftime("%Y%m%d")
from datetime import datetime, timedelta

today = datetime.today()
tmpfiles = []
while (1):
    if len(tmpfiles) == 10:
        break
    date_string = today.strftime("%Y%m%d") + ".export.CSV.zip"
    if date_string in file_list:
        tmpfiles.append(date_string)
    today -= timedelta(days=1)

In [3]:
import os.path
import urllib
import zipfile

if not os.path.isdir('data'):
    os.mkdir('data')

rdd = None
for download_file in tmpfiles:
    zipfilename = './data/' + download_file
    while not os.path.isfile(zipfilename):
        urllib.urlretrieve(url=gdelt_base_url + download_file,
                           filename=zipfilename)
    zf = zipfile.ZipFile(file=zipfilename, mode='r')
    for info in zf.infolist():
        data = zf.read(info.filename)
        tmprdd = sc.parallelize(data.split('\n')).map(lambda line: line.split('\t'))
        if rdd:
            rdd = sc.union([rdd, tmprdd])
        else:
            rdd = tmprdd
    zf.close()


# Reference
Graphframes Documents: [http://graphframes.github.io/user-guide.html#pagerank](http://graphframes.github.io/user-guide.html#pagerank)

Spark graphframes package: [https://spark-packages.org/package/graphframes/graphframes](https://spark-packages.org/package/graphframes/graphframes)